In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")

from skimage.morphology import label
from skimage.io import imread
from numpy import expand_dims

import os
import time
import sys


In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
#!python setup.py -q install

In [ ]:
from mrcnn.visualize import display_instances
from mrcnn.visualize import display_top_masks
from mrcnn.utils import extract_bboxes
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

In [ ]:
marks1 = pd.read_csv('/spell/ship_segmentations.csv') # Markers for ships
marks1.reset_index(drop=True,inplace=True)
marks_unique = marks1[marks1.EncodedPixels.notnull()]

marks=marks_unique[1:100]
marks.reset_index(drop=True,inplace=True)
images =list(marks.ImageId)
marks.head()


In [ ]:
test_marks=marks_unique[100:130]
test_marks.reset_index(drop=True,inplace=True)

test_images =list(test_marks.ImageId)
test_marks.head()

In [ ]:
marks[2:3].EncodedPixels


In [ ]:
TRAIN_DATA_PATH = '/spell/train'
TEST_DATA_PATH = '/spell/test'

IMAGE_WIDTH = 768
IMAGE_HEIGHT = 768
SHAPE = (IMAGE_WIDTH, IMAGE_HEIGHT)


In [ ]:

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


In [ ]:
class ShipDetection(Dataset):
    """Dataset class for training our dataset.
    """        
    # load the dataset definitions
    def load_dataset(self, images_list, marks_ann, dataset_dir,orig_height, orig_width):
         # Add classes
        self.add_class('ship', 1, 'Ship')
       # print(marks_ann)
        for i, fp in enumerate(images_list):            
            annotations=marks_ann.loc[marks_ann['ImageId'] == fp].EncodedPixels
            self.add_image('ship', image_id=i, path=os.path.join(dataset_dir, fp), 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
        
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        print(info)
        return info['path']

    #Load Image
    def load_image(self, image_id):
        info=self.image_info[image_id]
        #print(info)
        image = imread(info['path'])
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    #Load Mask
    def load_mask(self, image_id):
        #print(self.image_info)
        info=self.image_info[image_id]
        annotations = info['annotations']
#         print(image_id, annotations)
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                mask[:, :, i] = rle_decode(a,SHAPE)
                class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

In [ ]:
# prepare train set
train_set = ShipDetection()
train_set.load_dataset(images,marks, TRAIN_DATA_PATH, IMAGE_HEIGHT,IMAGE_WIDTH)
train_set.prepare()




In [ ]:
#Prepare test set
test_set = ShipDetection()
test_set.load_dataset(test_images,test_marks, TEST_DATA_PATH, IMAGE_HEIGHT,IMAGE_WIDTH)
test_set.prepare()

In [ ]:
# Test code to check if image loading, masking and boxing works properly on not
# load an image
image_id = 40
image = train_set.load_image(image_id)
#plt.title("Actual")
#plt.imshow(image)


# load image mask
mask, class_ids = train_set.load_mask(image_id)
print(class_ids)
# display image with masks and bounding boxes
#display_instances(image, bbox, mask, class_ids,  train_set.class_names,show_mask=True,show_bbox=True,title="Predicted",figsize=(8, 8))
display_top_masks(image, mask, class_ids, train_set.class_names, limit=1)

In [ ]:
class AirbusConfig(Config):

    # Setting other parameters...

    def __init__(self, num_classes):
        self.NAME = "airbus_cfg"
        self.NUM_CLASSES = num_classes
        self.STEPS_PER_EPOCH =500
        self.IMAGE_MAX_DIM=IMAGE_WIDTH
        self.IMAGE_MIN_DIM=IMAGE_HEIGHT
        
        
        super().__init__()
        
config = AirbusConfig(num_classes=2)
config.NUM_CLASSES
config.display()

In [ ]:
import mrcnn.model as modellib
from mrcnn.model import log
# Load and display using mrcnn model which is interally called all functions
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
        train_set, config, image_id= 25, use_mini_mask=False)
log("molded_image", image)
log("mask", mask)
display_instances(image, bbox, mask, class_ids, train_set.class_names,
                            show_bbox=False,show_mask=True)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# define the model
model = MaskRCNN(mode='training', model_dir='./', config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights('/spell/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
# train weights (output layers or 'heads')
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=2, layers='heads')

In [ ]:
# Code to get latest h5 file directory 
all_subdirs = [d for d in os.listdir('/spell/Mask_RCNN') if os.path.isdir(d)]
latest_subdir = max(all_subdirs, key=os.path.getmtime)
print(latest_subdir)

In [ ]:
# Predict for test dataset
# define the prediction configuration
class PredictionConfig(Config):

    # Setting other parameters...

    def __init__(self, num_classes):
        self.NAME = "airbus_cfg"
        self.NUM_CLASSES = 2
        self.GPU_COUNT = 1
        self.IMAGES_PER_GPU = 1
        self.STEPS_PER_EPOCH=100
        self.IMAGE_MAX_DIM=IMAGE_WIDTH
        self.IMAGE_MIN_DIM=IMAGE_HEIGHT
        
        super().__init__()
        
config_pred = PredictionConfig(2)
config_pred.display()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# create config
config_pred = PredictionConfig(num_classes=2)
# define the model
model_eval = MaskRCNN(mode='inference', model_dir='./', config=config_pred)
# load model weights
model_eval.load_weights('/spell/Mask_RCNN/' + latest_subdir+ '/mask_rcnn_airbus_cfg_0002.h5', by_name=True)



In [ ]:
def plot_actual_vs_predicted(dataset, model, cfg, image_id):
    # load image and mask 

    print(dataset.image_info[image_id]["path"])
    # load the image and mask
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    bbox = extract_bboxes(mask)  
    display_instances(image, bbox, mask,class_ids,dataset.class_names,title="Actual",figsize=(8, 8),show_mask=False)

    #Predicted image
    scaled_image = mold_image(image, cfg)
    # convert image into one sample
    sample = expand_dims(scaled_image, 0)
    # make prediction
    yhat = model.detect(sample, verbose=0)[0]

    # Visualize results
    r = yhat
    display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                       dataset.class_names, r['scores'],title="Predicted",figsize=(8, 8))


In [ ]:
# plot predictions for test dataset
plot_actual_vs_predicted(test_set, model_eval, config_pred,  image_id=1)

# Detect new images

In [ ]:

def detect_cells(path):
    # load image
    image = imread(path)
    
    plt.figure(figsize=(8,8))
    plt.title('Actual')
    plt.imshow(image)
    # convert pixel values (e.g. center)
    scaled_image = mold_image(image, config_pred)
    # convert image into one sample
    sample = expand_dims(scaled_image, 0)
    # make prediction
    yhat = model_eval.detect(sample, verbose=0)[0]
    r = yhat
    print(r['class_ids'])
    print( r['scores'])
    display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                train_set.class_names, r['scores'],title="Predicted",figsize=(8,8),show_mask=True)


In [ ]:
detect_cells("/spell/new/001eb2794.jpg")

In [ ]:
detect_cells("/spell/new/a97d55a5d.jpg")

In [ ]:
detect_cells("/spell/new/a9a01bb33.jpg")


In [ ]:
!python --version

In [ ]:
detect_cells("/spell/new/acb4aa56f.jpg")